In [68]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd
import numpy as np

In [69]:
data_set = pd.read_csv('../../data/TrueFX/EUR-USD/datos_procesados_OHLC_5T_EURUSD-2018-01.csv', index_col=0,
                      infer_datetime_format=True, parse_dates=True)

In [70]:
data_set.head()

,open,high,low,bid,ask
2018-01-02 00:00:00,1.20094,1.20094,1.20011,1.20015,1.20110
2018-01-02 00:05:00,1.20015,1.20062,1.20011,1.20047,1.20084
2018-01-02 00:10:00,1.20039,1.20098,1.20035,1.20082,1.20126
2018-01-02 00:15:00,1.20080,1.20097,1.20025,1.20035,1.20091
2018-01-02 00:20:00,1.20034,1.20057,1.20031,1.20049,1.20085


In [71]:
%run ../../code/PNLEstimatorWrapper.py
%run ../../code/EUtilities.py
EU = EUtilities()

In [72]:
window = 7
X, y_reg, bt = EU.build_dataset(data_set, window=window, bid_col='bid', binary_target=True, PNL=True)

In [73]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,-0.00076,0.00022
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,-0.00023,-0.00012
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,-0.00020,-0.00009
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00017


In [74]:
X['open'] = data_set.filter(X.index,axis=0).open
X['high'] = data_set.filter(X.index,axis=0).high
X['low'] = data_set.filter(X.index,axis=0).low

In [75]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1,open,high,low
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,-0.00076,0.00022,1.20048,1.20093,1.20045
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,-0.00023,-0.00012,1.20079,1.20163,1.20073
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,-0.00020,-0.00009,1.20129,1.20145,1.20129
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030,1.20143,1.20150,1.20134
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00017,1.20150,1.20150,1.20131


In [76]:
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, Activation, LSTM

In [77]:
X.shape, bt.shape

((6232, 12), (6232,))

In [99]:
X1 = np.array(X[[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])

In [101]:
X1 = np.array(X1)

In [103]:
scaler = MinMaxScaler(feature_range=(0, 1))
X1 = scaler.fit_transform(X1)

In [105]:
X1 = X1.reshape(X1.shape[0],X1.shape[1],1)
y = bt
X1.shape, y.shape

((6232, 10, 1), (6232,))

como es un modelo de multiclasificación toca utilizar categorical_crossentropy y para esta hay que pasar y en formato (n_filas,n_clases)

In [106]:
y2 = to_categorical(y,num_classes=4)

In [107]:
print(y2.shape)
y2

(6232, 4)


array([[ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       ..., 
       [ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.]])

In [114]:
X1.shape, y2.shape

((6232, 10, 1), (6232, 4))

In [158]:
model = Sequential()

model.add(LSTM(64, input_shape=(X1.shape[1],1), activation="relu"))
model.add(Dense(4, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

In [159]:
model.fit(X1[:4000], y2[:4000], epochs=100, batch_size=32)

Epoch 1/100
4000/4000 [==============================] - 1s - loss: 1.1413 - acc: 0.4285     
Epoch 2/100
4000/4000 [==============================] - 1s - loss: 1.0970 - acc: 0.4290     
Epoch 3/100
4000/4000 [==============================] - 1s - loss: 1.0951 - acc: 0.4235     
Epoch 4/100
4000/4000 [==============================] - 1s - loss: 1.0944 - acc: 0.4402     
Epoch 5/100
4000/4000 [==============================] - 1s - loss: 1.0939 - acc: 0.4305     
Epoch 6/100
4000/4000 [==============================] - 1s - loss: 1.0944 - acc: 0.4402     
Epoch 7/100
4000/4000 [==============================] - 1s - loss: 1.0941 - acc: 0.4290     
Epoch 8/100
4000/4000 [==============================] - 1s - loss: 1.0943 - acc: 0.4325     
Epoch 9/100
4000/4000 [==============================] - 1s - loss: 1.0943 - acc: 0.4312     
Epoch 10/100
4000/4000 [==============================] - 1s - loss: 1.0937 - acc: 0.4290     
Epoch 11/100
4000/4000 [==============================] - 1

4000/4000 [==============================] - 1s - loss: 1.0919 - acc: 0.4285     
Epoch 87/100
4000/4000 [==============================] - 1s - loss: 1.0917 - acc: 0.4222     
Epoch 88/100
4000/4000 [==============================] - 1s - loss: 1.0914 - acc: 0.4278     
Epoch 89/100
4000/4000 [==============================] - 1s - loss: 1.0917 - acc: 0.4297     
Epoch 90/100
4000/4000 [==============================] - 1s - loss: 1.0916 - acc: 0.4355     
Epoch 91/100
4000/4000 [==============================] - 1s - loss: 1.0917 - acc: 0.4390     
Epoch 92/100
4000/4000 [==============================] - 1s - loss: 1.0918 - acc: 0.4295     
Epoch 93/100
4000/4000 [==============================] - 1s - loss: 1.0916 - acc: 0.4243     
Epoch 94/100
4000/4000 [==============================] - 1s - loss: 1.0923 - acc: 0.4245     
Epoch 95/100
4000/4000 [==============================] - 1s - loss: 1.0916 - acc: 0.4400     
Epoch 96/100
4000/4000 [==============================] - 1s - 

In [182]:
model = Sequential()
model.add(LSTM(64, input_shape=(X1.shape[1],1), activation="relu"))
model.add(Dense(4, input_shape=(10,1), activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [183]:
model.fit(X1, y2, epochs=100, batch_size=32)

Epoch 1/100
6232/6232 [==============================] - 2s - loss: 1.1245 - acc: 0.4230     
Epoch 2/100
6232/6232 [==============================] - 1s - loss: 1.0810 - acc: 0.4296     
Epoch 3/100
6232/6232 [==============================] - 1s - loss: 1.0791 - acc: 0.4337     
Epoch 4/100
6232/6232 [==============================] - 1s - loss: 1.0798 - acc: 0.4358     
Epoch 5/100
6232/6232 [==============================] - 1s - loss: 1.0785 - acc: 0.4374     
Epoch 6/100
6232/6232 [==============================] - 1s - loss: 1.0780 - acc: 0.4414     
Epoch 7/100
6232/6232 [==============================] - 1s - loss: 1.0795 - acc: 0.4296     
Epoch 8/100
6232/6232 [==============================] - 1s - loss: 1.0788 - acc: 0.4361     
Epoch 9/100
6232/6232 [==============================] - 1s - loss: 1.0786 - acc: 0.4406     
Epoch 10/100
6232/6232 [==============================] - 2s - loss: 1.0793 - acc: 0.4389     
Epoch 11/100
6232/6232 [==============================] - 2

6232/6232 [==============================] - 1s - loss: 1.0773 - acc: 0.4302     
Epoch 88/100
6232/6232 [==============================] - 1s - loss: 1.0774 - acc: 0.4368     
Epoch 89/100
6232/6232 [==============================] - 1s - loss: 1.0769 - acc: 0.4381     
Epoch 90/100
6232/6232 [==============================] - 1s - loss: 1.0773 - acc: 0.4413     
Epoch 91/100
6232/6232 [==============================] - 1s - loss: 1.0770 - acc: 0.4387     
Epoch 92/100
6232/6232 [==============================] - 1s - loss: 1.0768 - acc: 0.4350     
Epoch 93/100
6232/6232 [==============================] - 1s - loss: 1.0767 - acc: 0.4434     
Epoch 94/100
6232/6232 [==============================] - 1s - loss: 1.0771 - acc: 0.4334     
Epoch 95/100
6232/6232 [==============================] - 1s - loss: 1.0767 - acc: 0.4373     
Epoch 96/100
6232/6232 [==============================] - 1s - loss: 1.0768 - acc: 0.4382     
Epoch 97/100
6232/6232 [==============================] - 1s - 

#### Se modifican las clases para poder utilizarlas con las RNN

In [64]:
# %load ../../code/PNLEstimatorWrapper.py
import pandas as pd
import numpy as np
from keras.utils import to_categorical

class PNLEstimatorWrapperRNN:
    
    def __init__(self, estimator, PNL_column, exclude_PNL_column_from_training=True):
        self.estimator = estimator
        self.PNL_column = PNL_column
        self.exclude_PNL_column_from_training = exclude_PNL_column_from_training
        
    def fit(self, X, y, n_epochs, n_batch_size):
        assert 'PNL_1' and 'PNL_0' in X.columns, "column "+self.PNL_column+" not in X dataframe"
        if self.exclude_PNL_column_from_training:
            X = np.array(X[[col for col in X.columns if col!=self.PNL_column[0] and col!=self.PNL_column[1]]])
            
        #para adaptar la X a la diension de las RNN
        X = X.reshape(X.shape[0],X.shape[1],1)
        self.estimator.fit(X,y, epochs=n_epochs, batch_size=n_batch_size)
        
    def predict(self, X):
        assert 'PNL_1' and 'PNL_0' in X.columns, "column "+self.PNL_column+" not in X dataframe"
        if self.exclude_PNL_column_from_training:
            X = X[[col for col in X.columns if col!=self.PNL_column[0] and col!=self.PNL_column[1]]]
            
        X = np.array(X)
        return self.estimator.predict(X)
    
    def score(self, X, y):
        pnl_1 = X.PNL_1
        pnl_0 = X.PNL_0
        pre = self.predict(X)
        r = sum((pre==1)*pnl_1 + (pre==0)*pnl_0)
        sell = sum(pre==0) + sum(pre==2)
        buy = sum(pre==1) + sum(pre==3)
        #percent_0 = np.mean(y==pre)*100
        percent_1 = np.mean((pre==1)*(y==1)) # si sube y gano
        percent_0 = np.mean((pre==0)*(y==0)) # si baja y gano
        percent_3 = np.mean((pre==3)*(y==3)) # si sube y pierdo
        percent_2 = np.mean((pre==2)*(y==2)) # si baja y pierdo
        
        count_ones, count_zeros = 0,0
        l_one, l_zero = [],[]
        for i in pre:
            if i==1:
                l_zero.append(count_zeros)
                count_zeros = 0
                count_ones += 1
            else:
                l_one.append(count_ones)
                count_zeros += 1
                count_ones = 0
        l_zero.append(count_zeros)
        l_one.append(count_ones)
        
        #return np.array([r,buy,sell,max(l_one),max(l_zero)])
        return np.array([r,buy,sell,max(l_one),max(l_zero),percent_1,percent_0,percent_3,percent_2]), pre
        
    def get_params(self, deep=False):
        return {"PNL_column": self.PNL_column,
                "exclude_PNL_column_from_training": self.exclude_PNL_column_from_training,
                "estimator": self.estimator}

In [65]:
model = Sequential()

model.add(LSTM(64, input_shape=(10,1), activation="sigmoid"))
model.add(Dense(4, activation="sigmoid"))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

In [66]:
est = PNLEstimatorWrapperRNN(model, PNL_column=['PNL_0','PNL_1'])

In [67]:
est.fit(X, y2, n_epochs=200, n_batch_size=32)

Epoch 1/200
6232/6232 [==============================] - 1s - loss: 1.1039 - acc: 0.4339     
Epoch 2/200
6232/6232 [==============================] - 1s - loss: 1.0792 - acc: 0.4369     
Epoch 3/200
6232/6232 [==============================] - 2s - loss: 1.0795 - acc: 0.4430     
Epoch 4/200
6232/6232 [==============================] - 1s - loss: 1.0794 - acc: 0.4381     
Epoch 5/200
6232/6232 [==============================] - 1s - loss: 1.0794 - acc: 0.4288     
Epoch 6/200
6232/6232 [==============================] - 1s - loss: 1.0794 - acc: 0.4413     
Epoch 7/200
6232/6232 [==============================] - 1s - loss: 1.0796 - acc: 0.4318     
Epoch 8/200
6232/6232 [==============================] - 1s - loss: 1.0797 - acc: 0.4352     
Epoch 9/200
6232/6232 [==============================] - 1s - loss: 1.0793 - acc: 0.4350     
Epoch 10/200
6232/6232 [==============================] - 2s - loss: 1.0796 - acc: 0.4331     
Epoch 11/200
6232/6232 [==============================] - 1

6232/6232 [==============================] - 2s - loss: 1.0783 - acc: 0.4395     
Epoch 88/200
6232/6232 [==============================] - 2s - loss: 1.0788 - acc: 0.4312     
Epoch 89/200
6232/6232 [==============================] - 2s - loss: 1.0788 - acc: 0.4368     
Epoch 90/200
6232/6232 [==============================] - 2s - loss: 1.0787 - acc: 0.4401     
Epoch 91/200
6232/6232 [==============================] - 2s - loss: 1.0788 - acc: 0.4353     
Epoch 92/200
6232/6232 [==============================] - 2s - loss: 1.0789 - acc: 0.4289     
Epoch 93/200
6232/6232 [==============================] - 2s - loss: 1.0787 - acc: 0.4387     
Epoch 94/200
6232/6232 [==============================] - 2s - loss: 1.0790 - acc: 0.4355     
Epoch 95/200
6232/6232 [==============================] - 2s - loss: 1.0790 - acc: 0.4361     
Epoch 96/200
6232/6232 [==============================] - 2s - loss: 1.0789 - acc: 0.4331     
Epoch 97/200
6232/6232 [==============================] - 2s - 

6232/6232 [==============================] - 2s - loss: 1.0789 - acc: 0.4341     
Epoch 173/200
6232/6232 [==============================] - 2s - loss: 1.0790 - acc: 0.4374     
Epoch 174/200
6232/6232 [==============================] - 2s - loss: 1.0785 - acc: 0.4251     
Epoch 175/200
6232/6232 [==============================] - 2s - loss: 1.0786 - acc: 0.4393     
Epoch 176/200
6232/6232 [==============================] - 2s - loss: 1.0787 - acc: 0.4352     
Epoch 177/200
6232/6232 [==============================] - 2s - loss: 1.0785 - acc: 0.4427     
Epoch 178/200
6232/6232 [==============================] - 2s - loss: 1.0790 - acc: 0.4363     
Epoch 179/200
6232/6232 [==============================] - 2s - loss: 1.0788 - acc: 0.4366     
Epoch 180/200
6232/6232 [==============================] - 2s - loss: 1.0786 - acc: 0.4403     
Epoch 181/200
6232/6232 [==============================] - 2s - loss: 1.0792 - acc: 0.4331     
Epoch 182/200
6232/6232 [=============================

In [34]:
# %load ../../code/v_split.py
# Se crea un generador "v_split" para utilizar como método de validación cruzada
def v_split(X, n_bdtrain, n_bdtest, mday):

    """"
    Hace un particionado del dataset, para tomar n_bdtrain días para entrenar
    y n_bdtest para probar, además, mday representa el paso de día a correr.
    X, dataframe, se necesita el indice de este para separar por días.
    n_bdtrain, número de bussines day utilizados para train.
    n_bdtest, número de bussines day utilizados para test.
    mday, días a correr para cada validación.
    
    """
    import pandas as pd
    import numpy as np
    from datetime import date
    start_day = 0
    
    #Divide el data set según días de train, test y cuanto se va moviendo
    bussines_day = []
    dates = pd.unique(X.index.date) #saco las fechas para luego tomar solo año-mes-día

    for i in dates: bussines_day.append(date.__format__(i,'%Y-%m-%d')) #lista de los bussines day
    
    intervals = []
    count = 0
    for i in bussines_day:
        f = len(X[i])-1 +count
        intervals.append([count,f])
        count = f+1
    
    for i in range(len(intervals)-n_bdtrain):
        yield(np.arange(intervals[start_day:start_day+n_bdtrain][0][0],
                        intervals[start_day:start_day+n_bdtrain][n_bdtrain-1][1]+1),
              np.arange(intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][0][0],
                        intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][n_bdtest-1][1]+1))
    
        start_day += mday
        if start_day+n_bdtest > len(intervals)-n_bdtrain:
            break

In [47]:
# %load ../../code/step_validation.py
def step_validationRNN(estimator, X, y, cv, n_epochs, n_batch_size):
    '''
    Recibe el estimador,X,y, y un generador cv con el cual hace la validación
    dependiendo que la configuración que este tenga
    '''
    import numpy as np
    result = []
    for index in cv:
        estimator.fit(X.iloc[index[0]], y[index[0]], epochs=n_epochs, batch_size=n_batch_size)
        result.append(estimator.score(X.iloc[index[1]], y[index[1]]))
    return np.array(result)